Load all libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import random

In [188]:
pd.set_option('display.max_rows', 500)

Install Geopy to calculate distances between points

In [10]:
!pip install geopy

In [31]:
from geopy import distance, geocoders, Nominatim
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut, GeocoderServiceError
from geopy.extra.rate_limiter import RateLimiter

Load all Datasets saved on csv files

In [2]:
df_mad_houses=pd.read_csv('df_mad_houses.csv')

In [ ]:
df_mad_over100k=pd.read_csv('df_mad_over100k.csv')

In [ ]:
df_transport_stops=pd.read_csv('transport_stops_madrid.csv')

In [105]:
df_mad_postcode_latlon= pd.read_csv('latlons_by_postcode_madrid.csv')

In [133]:
df_mad_houses=df_mad_houses.drop(columns=['Unnamed: 0','codigopostalid','lon','lat'])

In [135]:
df_mad_houses

,prices,post_code,rooms,bathrooms,size,poblacion,codigopostalid,lat,lon,latlon_merged
0,112.062€,28031,1 hab,1 wc,49 m²,Villa Vallecas,28031,40.377436,-3.624158,"(40.377435628148554, -3.624158084914701)"
1,137.000€,28031,1 hab,1 wc,32 m²,Villa Vallecas,28031,40.377436,-3.624158,"(40.377435628148554, -3.624158084914701)"
2,100.000€,28031,3 hab,1 wc,48 m²,Villa Vallecas,28031,40.377436,-3.624158,"(40.377435628148554, -3.624158084914701)"
3,100.000€,28031,3 hab,1 wc,48 m²,Villa Vallecas,28031,40.377436,-3.624158,"(40.377435628148554, -3.624158084914701)"
4,135.000€,28031,1 hab,1 wc,49 m²,Villa Vallecas,28031,40.377436,-3.624158,"(40.377435628148554, -3.624158084914701)"
...,...,...,...,...,...,...,...,...,...,...
26660,160.000€,28193,4 hab,1 wc,184 m²,Cervera de Buitrago,28193,43.420435,-3.529145,"(43.420435, -3.5291447)"
26661,360.000€,28512,4 hab,2 wc,189 m²,Villar del Olmo,28512,40.343467,-3.241504,"(40.34346737484606, -3.2415042886969347)"
26662,260.000€,28595,4 hab,3 wc,200 m²,Estremera,28595,43.420435,-3.529145,"(43.420435, -3.5291447)"
26663,145.000€,28595,3 hab,3 wc,200 m²,Estremera,28595,43.420435,-3.529145,"(43.420435, -3.5291447)"


In [139]:
df_mad_houses['poblacion'].value_counts()

San Blas                807
Alcalá de Henares       756
Las Rozas de Madrid     613
Torrejón de Ardoz       602
Justicia                599
                       ... 
Valdaracete               1
Cervera de Buitrago       1
Soto del Real             1
Rozas de Puerto Real      1
Villar del Olmo           1
Name: poblacion, Length: 196, dtype: int64

Change the postcode file to include city and country to be able to make searches to the Nominatim API and get the latlons of each postcode

In [82]:
df_mad_postcode_latlon['address']=list(zip(df_mad_postcode_latlon['codigopostalid'],df_mad_postcode_latlon['city'],df_mad_postcode_latlon['country']))

In [111]:
df_mad_postcode_latlon=df_mad_postcode_latlon.drop(columns=['Unnamed: 0','full_address','location'])

In [269]:
user_agent = 'palvgoya@hotmail.com'
geolocator = Nominatim(user_agent=user_agent)

geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
df_mad_postcode_latlon['location'] = df_mad_postcode_latlon['address'].apply(geocode)

df_mad_postcode_latlon['location'] = df_mad_postcode_latlon['location'].apply(lambda loc: tuple(loc.point) if loc else None)

KeyError: 'address'

In [120]:
df_mad_postcode_latlon[['lat', 'lon', 'alt']] = pd.DataFrame(df_mad_postcode_latlon['location'].tolist(), index=df_mad_postcode_latlon.index)  

In [122]:
df_mad_postcode_latlon['latlon_merged']=list(zip(df_mad_postcode_latlon['lat'],df_mad_postcode_latlon['lon']))

In [124]:
df_mad_postcode_latlon.to_csv('latlons_by_postcode_madrid.csv')

In [125]:
df_mad_postcode_latlon

,codigopostalid,city,country,address,location,lat,lon,alt,latlon_merged
0,28001,madrid,spain,"(28001, madrid, spain)","(40.4271819, -3.6872776666649734, 0.0)",40.427182,-3.687278,0.0,"(40.4271819, -3.6872776666649734)"
1,28002,madrid,spain,"(28002, madrid, spain)","(40.44436359786106, -3.671972654609913, 0.0)",40.444364,-3.671973,0.0,"(40.44436359786106, -3.671972654609913)"
2,28003,madrid,spain,"(28003, madrid, spain)","(40.44196013762609, -3.7053510586159732, 0.0)",40.441960,-3.705351,0.0,"(40.44196013762609, -3.7053510586159732)"
3,28004,madrid,spain,"(28004, madrid, spain)","(40.427525349999996, -3.6950212296048797, 0.0)",40.427525,-3.695021,0.0,"(40.427525349999996, -3.6950212296048797)"
4,28005,madrid,spain,"(28005, madrid, spain)","(40.409289037338226, -3.710041004754097, 0.0)",40.409289,-3.710041,0.0,"(40.409289037338226, -3.710041004754097)"
5,28006,madrid,spain,"(28006, madrid, spain)","(40.43383533987043, -3.680541208403145, 0.0)",40.433835,-3.680541,0.0,"(40.43383533987043, -3.680541208403145)"
6,28007,madrid,spain,"(28007, madrid, spain)","(40.4046034, -3.6834474261815178, 0.0)",40.404603,-3.683447,0.0,"(40.4046034, -3.6834474261815178)"
7,28008,madrid,spain,"(28008, madrid, spain)","(40.42691291585265, -3.7198520527710333, 0.0)",40.426913,-3.719852,0.0,"(40.42691291585265, -3.7198520527710333)"
8,28009,madrid,spain,"(28009, madrid, spain)","(40.41956994587277, -3.6757184630348814, 0.0)",40.419570,-3.675718,0.0,"(40.41956994587277, -3.6757184630348814)"
9,28010,madrid,spain,"(28010, madrid, spain)","(40.432961272457646, -3.6994969982495944, 0.0)",40.432961,-3.699497,0.0,"(40.432961272457646, -3.6994969982495944)"


In [126]:
df_mad_postcode_latlon=df_mad_postcode_latlon.drop(columns=['city','country','address','location','alt'])

Merge the houses with the latlons file to get latlons of each house by postcode

In [134]:
df_mad_houses=df_mad_houses.merge(df_mad_postcode_latlon,left_on='post_code',right_on='codigopostalid')

Cleaning the transport stops file to use only the columns we need

In [63]:
df_transport_stops=transport_df= df_transport_stops.drop(columns=['stop_id', 'stop_code', 'stop_name', 'stop_desc', 'zone_id','stop_url', 'location_type', 'parent_station', 'stop_timezone', 'wheelchair_boarding'])

In [64]:
df_transport_stops['stop_loc'] = list(zip(df_transport_stops['stop_lat'], df_transport_stops['stop_lon']))

Calculating closest transport stop to center point of each neighbourhood

In [206]:
dist_transport=[]

for a in range(0,26665):
    point = df_mad_houses['latlon_merged'][a]
    dist = df_transport_stops['stop_loc'].apply(lambda x: distance.geodesic(x, point).m).min()
    dist_transport.append(dist)

In [207]:
df_mad_houses['transport']=dist_transport

In [208]:
len(dist_transport)

26665

In [195]:
df_mad_houses.loc[df_mad_houses['poblacion'] == 'Talamanca de Jarama', ['lat','lon','latlon_merged']] = '40.7460256','-3.5129914','(40.7460256, -3.5129914)'
df_mad_houses.loc[df_mad_houses['poblacion'] == 'Belmonte de Tajo', ['lat','lon','latlon_merged']] = '40.1336305','-3.3398654','(40.1336305, -3.3398654)'
df_mad_houses.loc[df_mad_houses['poblacion'] == 'Carabaña', ['lat','lon','latlon_merged']] = '40.256626','-3.2347206','(40.256626, -3.2347206)'
df_mad_houses.loc[df_mad_houses['poblacion'] == 'Robledo de Chavela', ['lat','lon','latlon_merged']] = '40.5009201','-4.2380999','(40.5009201, -4.2380999)'
df_mad_houses.loc[df_mad_houses['poblacion'] == 'La Cabrera', ['lat','lon','latlon_merged']] = '40.8651738','-3.6125468','(40.8651738, -3.6125468)'
df_mad_houses.loc[df_mad_houses['poblacion'] == 'Ribatejada', ['lat','lon','latlon_merged']] = '40.6664949','-3.3919178','(40.6664949, -3.3919178)'
df_mad_houses.loc[df_mad_houses['poblacion'] == 'Villa del Prado', ['lat','lon','latlon_merged']] = '40.2764752','-4.3048032','(40.2764752, -4.3048032)'
df_mad_houses.loc[df_mad_houses['poblacion'] == 'Torrelaguna', ['lat','lon','latlon_merged']] = '40.8277273','-3.538293','(40.8277273, -3.538293)'
df_mad_houses.loc[df_mad_houses['poblacion'] == 'Santa María de la Alameda', ['lat','lon','latlon_merged']] = '40.5958248','-4.2579208','(40.5958248, -4.2579208)'
df_mad_houses.loc[df_mad_houses['poblacion'] == 'Valdemanco', ['lat','lon','latlon_merged']] = '40.863336950000004','-3.6624598591227406','(40.863336950000004, -3.6624598591227406)'
df_mad_houses.loc[df_mad_houses['poblacion'] == 'El Vellón', ['lat','lon','latlon_merged']] = '40.7677965','-3.5820099','(40.7677965, -3.5820099)'
df_mad_houses.loc[df_mad_houses['poblacion'] == 'Aldea del Fresno', ['lat','lon','latlon_merged']] = '40.3215212','-4.19936','(40.3215212, -4.19936)'
df_mad_houses.loc[df_mad_houses['poblacion'] == 'Cubas de la Sagra', ['lat','lon','latlon_merged']] = '40.1908352','-3.8378366', '(40.1908352, -3.8378366)'
df_mad_houses.loc[df_mad_houses['poblacion'] == 'Perales de Tajuña', ['lat','lon','latlon_merged']] = '40.2325718','-3.3517265', '(40.2325718, -3.3517265)'
df_mad_houses.loc[df_mad_houses['poblacion'] == 'Valdilecha', ['lat','lon','latlon_merged']] = '40.2926854','-3.3002096', '(40.2926854, -3.3002096)'
df_mad_houses.loc[df_mad_houses['poblacion'] == 'Bustarviejo', ['lat','lon','latlon_merged']] = '40.8586659','-3.7101535', '(40.8586659, -3.7101535)'
df_mad_houses.loc[df_mad_houses['poblacion'] == 'Getafe', ['lat','lon','latlon_merged']] = '40.3081807','-3.7302679', '(40.3081807, -3.7302679)'

In [211]:
df_mad_houses=df_mad_houses[df_mad_houses['transport']<=50000]

In [205]:
df_mad_houses['latlon_merged']=list(zip(df_mad_houses['lat'],df_mad_houses['lon']))

In [357]:
df_mad_houses.to_csv('df_mad_houses.csv')

Calculate distance of each house to closest school

In [214]:
df_schools=pd.read_csv('Adresses_Schools_Mad.csv')

In [297]:
geolocator = geocoders.ArcGIS()

#geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
df_schools['location'] = df_schools['address'].apply(geolocator.geocode,timeout=10)

#df_hospitals['location'] = df_mad_postcode_latlon['location'].apply(lambda loc: tuple(loc.point) if loc else None)

KeyError: 'location'

In [316]:
df_schools[['a','latlon']]= pd.DataFrame(df_schools['location'].to_list(),index=df_schools.index)

In [319]:
df_schools=df_schools.drop(columns=['a','address','random1','random2','Domicilio','Código'])

In [321]:
df_schools.to_csv('schools_mad_final.csv')

In [346]:
dist_school=[]

for a in range(0,26600):
    point = df_mad_houses['latlon_merged'][a]
    dist = df_schools['latlon'].apply(lambda x: distance.geodesic(x, point).m).min()
    dist_school.append(dist)

KeyError: 26599

In [347]:
dist_school

[201.2356742507121,
 201.2356742507121,
 201.2356742507121,
 201.2356742507121,
 201.2356742507121,
 201.2356742507121,
 201.2356742507121,
 201.2356742507121,
 201.2356742507121,
 201.2356742507121,
 201.2356742507121,
 201.2356742507121,
 201.2356742507121,
 201.2356742507121,
 201.2356742507121,
 201.2356742507121,
 201.2356742507121,
 201.2356742507121,
 201.2356742507121,
 201.2356742507121,
 201.2356742507121,
 201.2356742507121,
 201.2356742507121,
 201.2356742507121,
 201.2356742507121,
 201.2356742507121,
 201.2356742507121,
 201.2356742507121,
 201.2356742507121,
 201.2356742507121,
 201.2356742507121,
 201.2356742507121,
 201.2356742507121,
 201.2356742507121,
 201.2356742507121,
 201.2356742507121,
 201.2356742507121,
 201.2356742507121,
 201.2356742507121,
 201.2356742507121,
 201.2356742507121,
 201.2356742507121,
 201.2356742507121,
 201.2356742507121,
 201.2356742507121,
 201.2356742507121,
 201.2356742507121,
 201.2356742507121,
 201.2356742507121,
 201.2356742507121,


In [348]:
df_mad_houses['school']=dist_school

In [349]:
df_mad_houses

,prices,post_code,rooms,bathrooms,size,poblacion,lat,lon,latlon_merged,transport,school
0,112.062€,28031,1 hab,1 wc,49 m²,Villa Vallecas,40.3774,-3.62416,"(40.377435628148554, -3.624158084914701)",324.842708,201.235674
1,137.000€,28031,1 hab,1 wc,32 m²,Villa Vallecas,40.3774,-3.62416,"(40.377435628148554, -3.624158084914701)",324.842708,201.235674
2,100.000€,28031,3 hab,1 wc,48 m²,Villa Vallecas,40.3774,-3.62416,"(40.377435628148554, -3.624158084914701)",324.842708,201.235674
3,100.000€,28031,3 hab,1 wc,48 m²,Villa Vallecas,40.3774,-3.62416,"(40.377435628148554, -3.624158084914701)",324.842708,201.235674
4,135.000€,28031,1 hab,1 wc,49 m²,Villa Vallecas,40.3774,-3.62416,"(40.377435628148554, -3.624158084914701)",324.842708,201.235674
...,...,...,...,...,...,...,...,...,...,...,...
26594,175.000€,28721,4 hab,2 wc,156 m²,Cabanillas de la Sierra,40.818,-3.60548,"(40.81798043643363, -3.6054796201378334)",23994.341683,1790.021114
26595,175.000€,28721,4 hab,2 wc,156 m²,Cabanillas de la Sierra,40.818,-3.60548,"(40.81798043643363, -3.6054796201378334)",23994.341683,1790.021114
26596,329.000€,28791,3 hab,2 wc,150 m²,Soto del Real,40.7526,-3.77991,"(40.752559745929474, -3.7799147561052875)",11952.570673,307.756355
26597,360.000€,28512,4 hab,2 wc,189 m²,Villar del Olmo,40.3435,-3.2415,"(40.34346737484606, -3.2415042886969347)",18058.216754,899.348506


Calcualte distance of each house to closest health centre

In [298]:
df_hospitals=pd.read_csv('salud_madrid.csv')

In [293]:
df_hospitals['address']= df_hospitals['address'].str.replace('[', ',')
df_hospitals['address']= df_hospitals['address'].str.replace(']', '')

In [294]:
df_hospitals=df_hospitals.drop(columns=['direccion','random1','random2'])

In [299]:
geolocator = geocoders.ArcGIS()

df_hospitals['location'] = df_hospitals['address'].apply(geolocator.geocode,timeout=10)


KeyError: 'location'

In [286]:
from geopy import geocoders
g = geocoders.ArcGIS()

#create an input address string
#you can also build this by reading from an input database and building a string

inputAddress = 'Prof. Martin Lagos, s/n  ,28040, Madrid, Spain' 

#do the geocode
#location = g.geocode(inputAddress, timeout=10)
location= inputAddress.apply(g.geocode,timeout=10)

#some things you can get from the result
#print(location.latitude, location.longitude)

print(location)


AttributeError: 'str' object has no attribute 'apply'

In [303]:
df_hospitals[['place','latlon']] = pd.DataFrame(df_hospitals['location'].tolist(), index=df_hospitals.index)  

In [306]:
df_hospitals.to_csv('salud_madrid_final.csv')

In [354]:
dist_health_centre=[]

for a in range(0,26599):
    point = df_mad_houses['latlon_merged'][a]
    dist = df_hospitals['latlon'].apply(lambda x: distance.geodesic(x, point).m).min()
    dist_health_centre.append(dist)

In [355]:
df_mad_houses['health_centre']=dist_health_centre

In [356]:
df_mad_houses

,prices,post_code,rooms,bathrooms,size,poblacion,lat,lon,latlon_merged,transport,school,health_centre
0,112.062€,28031,1 hab,1 wc,49 m²,Villa Vallecas,40.3774,-3.62416,"(40.377435628148554, -3.624158084914701)",324.842708,201.235674,892.420899
1,137.000€,28031,1 hab,1 wc,32 m²,Villa Vallecas,40.3774,-3.62416,"(40.377435628148554, -3.624158084914701)",324.842708,201.235674,892.420899
2,100.000€,28031,3 hab,1 wc,48 m²,Villa Vallecas,40.3774,-3.62416,"(40.377435628148554, -3.624158084914701)",324.842708,201.235674,892.420899
3,100.000€,28031,3 hab,1 wc,48 m²,Villa Vallecas,40.3774,-3.62416,"(40.377435628148554, -3.624158084914701)",324.842708,201.235674,892.420899
4,135.000€,28031,1 hab,1 wc,49 m²,Villa Vallecas,40.3774,-3.62416,"(40.377435628148554, -3.624158084914701)",324.842708,201.235674,892.420899
...,...,...,...,...,...,...,...,...,...,...,...,...
26594,175.000€,28721,4 hab,2 wc,156 m²,Cabanillas de la Sierra,40.818,-3.60548,"(40.81798043643363, -3.6054796201378334)",23994.341683,1790.021114,5226.267593
26595,175.000€,28721,4 hab,2 wc,156 m²,Cabanillas de la Sierra,40.818,-3.60548,"(40.81798043643363, -3.6054796201378334)",23994.341683,1790.021114,5226.267593
26596,329.000€,28791,3 hab,2 wc,150 m²,Soto del Real,40.7526,-3.77991,"(40.752559745929474, -3.7799147561052875)",11952.570673,307.756355,636.752194
26597,360.000€,28512,4 hab,2 wc,189 m²,Villar del Olmo,40.3435,-3.2415,"(40.34346737484606, -3.2415042886969347)",18058.216754,899.348506,11851.252744


Load past data to start training and estimate neighbourhood prices per m2 in 2030

In [4]:
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split

In [424]:
df_past_mad= pd.read_csv('dataset_16_19_mad_3.csv')
df_past_mad.head(10)

,year,neighbourhood,transactions,avg_eur_m2,avgsize
0,01-01-2016,Alcalá de Henares,1612.0,1427.0,94.43
1,01-01-2016,Alcobendas,804.0,2394.0,112.49
2,01-01-2016,Alcorcón,1096.0,1696.0,95.73
3,01-01-2016,Aranjuez,510.0,1373.0,95.26
4,01-01-2016,Arganda del Rey,537.0,1439.0,104.30
5,01-01-2016,Boadilla del Monte,440.0,2587.0,129.51
6,01-01-2016,Collado Villalba,594.0,1597.0,100.82
7,01-01-2016,Colmenar Viejo,316.0,1888.0,92.34
8,01-01-2016,Coslada,467.0,1812.0,97.32
9,01-01-2016,Fuenlabrada,1092.0,1500.0,92.20


In [425]:
pd.to_datetime(df_past_mad['year'])

0     2016-01-01
1     2016-01-01
2     2016-01-01
3     2016-01-01
4     2016-01-01
         ...    
623   2019-01-01
624   2019-01-01
625   2019-01-01
626   2019-01-01
627   2019-01-01
Name: year, Length: 628, dtype: datetime64[ns]

In [ ]:
df_predictions_past=pd.Dataframe()

df_predictions_past['year']=2020-01-01
df_predictions_past['neighbourhood']=df_past_mad['neighbourhood']
df_predictions_past['transactions']=df_past_mad['transactions']
df_predictions_past['avgsize']=df_past_mad['avgsize']

In [144]:
df_past_mad['neighbourhood'].value_counts()

M-Timón              20
M-RíosRosas          20
M-SanIsidro          20
M-SanJuanBautista    20
M-Almendrales        20
                     ..
M-Delicias           20
Rivas-Vaciamadrid    20
M-Fontarrón          20
M-Ángeles            20
M-Águilas            20
Name: neighbourhood, Length: 157, dtype: int64

In [145]:
from sklearn.preprocessing import LabelEncoder

# creating instance of labelencoder
labelencoder = LabelEncoder()
# Assigning numerical values and storing in another column
df_past_mad['neighbourhood_cat'] = labelencoder.fit_transform(df_past_mad['neighbourhood'])
df_past_mad

,year,neighbourhood,size,transactions,avg_eur_m2,avgsize,gdp,unempl_rate,neighbourhood_cat
0,2016,M-Palacio,under 40 m2,41,3268.58,77.55,NaN,NaN,97
1,2016,M-Palacio,40 a 60 m2,49,3268.58,77.55,NaN,NaN,97
2,2016,M-Palacio,60 a 80 m2,33,3268.58,77.55,NaN,NaN,97
3,2016,M-Palacio,80 a 100 m2,23,3268.58,77.55,NaN,NaN,97
4,2016,M-Palacio,over 100 m2,32,3268.58,77.55,NaN,NaN,97
...,...,...,...,...,...,...,...,...,...
3135,2019,Villaviciosa de Odón,under 40 m2,7,2589,96,NaN,NaN,156
3136,2019,Villaviciosa de Odón,40 a 60 m2,49,2589,96,NaN,NaN,156
3137,2019,Villaviciosa de Odón,60 a 80 m2,87,2589,96,NaN,NaN,156
3138,2019,Villaviciosa de Odón,80 a 100 m2,58,2589,96,NaN,NaN,156


In [146]:
# Assigning numerical values and storing in another column
df_past_mad['year_cat'] = labelencoder.fit_transform(df_past_mad['year'])
df_past_mad

,year,neighbourhood,size,transactions,avg_eur_m2,avgsize,gdp,unempl_rate,neighbourhood_cat,year_cat
0,2016,M-Palacio,under 40 m2,41,3268.58,77.55,NaN,NaN,97,0
1,2016,M-Palacio,40 a 60 m2,49,3268.58,77.55,NaN,NaN,97,0
2,2016,M-Palacio,60 a 80 m2,33,3268.58,77.55,NaN,NaN,97,0
3,2016,M-Palacio,80 a 100 m2,23,3268.58,77.55,NaN,NaN,97,0
4,2016,M-Palacio,over 100 m2,32,3268.58,77.55,NaN,NaN,97,0
...,...,...,...,...,...,...,...,...,...,...
3135,2019,Villaviciosa de Odón,under 40 m2,7,2589,96,NaN,NaN,156,3
3136,2019,Villaviciosa de Odón,40 a 60 m2,49,2589,96,NaN,NaN,156,3
3137,2019,Villaviciosa de Odón,60 a 80 m2,87,2589,96,NaN,NaN,156,3
3138,2019,Villaviciosa de Odón,80 a 100 m2,58,2589,96,NaN,NaN,156,3


In [147]:
# Assigning numerical values and storing in another column
df_past_mad['size_cat'] = labelencoder.fit_transform(df_past_mad['size'])
df_past_mad

,year,neighbourhood,size,transactions,avg_eur_m2,avgsize,gdp,unempl_rate,neighbourhood_cat,year_cat,size_cat
0,2016,M-Palacio,under 40 m2,41,3268.58,77.55,NaN,NaN,97,0,4
1,2016,M-Palacio,40 a 60 m2,49,3268.58,77.55,NaN,NaN,97,0,0
2,2016,M-Palacio,60 a 80 m2,33,3268.58,77.55,NaN,NaN,97,0,1
3,2016,M-Palacio,80 a 100 m2,23,3268.58,77.55,NaN,NaN,97,0,2
4,2016,M-Palacio,over 100 m2,32,3268.58,77.55,NaN,NaN,97,0,3
...,...,...,...,...,...,...,...,...,...,...,...
3135,2019,Villaviciosa de Odón,under 40 m2,7,2589,96,NaN,NaN,156,3,4
3136,2019,Villaviciosa de Odón,40 a 60 m2,49,2589,96,NaN,NaN,156,3,0
3137,2019,Villaviciosa de Odón,60 a 80 m2,87,2589,96,NaN,NaN,156,3,1
3138,2019,Villaviciosa de Odón,80 a 100 m2,58,2589,96,NaN,NaN,156,3,2


In [112]:
df_past_mad.loc[df_past_mad['size'] == 'under 40 m2', 'under40'] = 1
df_past_mad.loc[df_past_mad['size'] != 'under 40 m2', 'under40'] = 0
df_past_mad.loc[df_past_mad['size'] == '40 a 60 m2', '40_60'] = 1
df_past_mad.loc[df_past_mad['size'] != '40 a 60 m2', '40_60'] = 0
df_past_mad.loc[df_past_mad['size'] == '60 a 80 m2', '60_80'] = 1
df_past_mad.loc[df_past_mad['size'] != '60 a 80 m2', '60_80'] = 0
df_past_mad.loc[df_past_mad['size'] == '80 a 100 m2', '80_100'] = 1
df_past_mad.loc[df_past_mad['size'] != '80 a 100 m2', '80_100'] = 0
df_past_mad.loc[df_past_mad['size'] == 'over 100 m2', 'over100'] = 1
df_past_mad.loc[df_past_mad['size'] != 'over 100 m2', 'over100'] = 0


df_past_mad

,year,neighbourhood,size,transactions,avg_eur_m2,avgsize,gdp,unempl_rate,under40,40_60,60_80,80_100,over100
0,2016,011. Palacio,under 40 m2,41,3268.58,77.55,NaN,NaN,1.0,0.0,0.0,0.0,0.0
1,2016,011. Palacio,40 a 60 m2,49,3268.58,77.55,NaN,NaN,0.0,1.0,0.0,0.0,0.0
2,2016,011. Palacio,60 a 80 m2,33,3268.58,77.55,NaN,NaN,0.0,0.0,1.0,0.0,0.0
3,2016,011. Palacio,80 a 100 m2,23,3268.58,77.55,NaN,NaN,0.0,0.0,0.0,1.0,0.0
4,2016,011. Palacio,over 100 m2,32,3268.58,77.55,NaN,NaN,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3135,2019,Villaviciosa de Odón,under 40 m2,7,2589,96,NaN,NaN,1.0,0.0,0.0,0.0,0.0
3136,2019,Villaviciosa de Odón,40 a 60 m2,49,2589,96,NaN,NaN,0.0,1.0,0.0,0.0,0.0
3137,2019,Villaviciosa de Odón,60 a 80 m2,87,2589,96,NaN,NaN,0.0,0.0,1.0,0.0,0.0
3138,2019,Villaviciosa de Odón,80 a 100 m2,58,2589,96,NaN,NaN,0.0,0.0,0.0,1.0,0.0


In [113]:
df_past_mad.loc[df_past_mad['year'] == 2016, 2016] = 1
df_past_mad.loc[df_past_mad['year'] != 2016, 2016] = 0
df_past_mad.loc[df_past_mad['year'] == 2017, 2017] = 1
df_past_mad.loc[df_past_mad['year'] != 2017, 2017] = 0
df_past_mad.loc[df_past_mad['year'] == 2018, 2018] = 1
df_past_mad.loc[df_past_mad['year'] != 2018, 2018] = 0
df_past_mad.loc[df_past_mad['year'] == 2019, 2019] = 1
df_past_mad.loc[df_past_mad['year'] != 2019, 2019] = 0


In [148]:
df_past_mad['transactions'] = df_past_mad['transactions'].replace('-','0')
df_past_mad['avgsize'] = df_past_mad['avgsize'].replace('-','0')
df_past_mad['avgsize'] = df_past_mad['avgsize'].replace(',','.')
df_past_mad['avg_eur_m2'] = df_past_mad['avg_eur_m2'].replace('-','0')
df_past_mad['avg_eur_m2'] = df_past_mad['avg_eur_m2'].replace(',','.')

In [149]:
df_past_mad

,year,neighbourhood,size,transactions,avg_eur_m2,avgsize,gdp,unempl_rate,neighbourhood_cat,year_cat,size_cat
0,2016,M-Palacio,under 40 m2,41,3268.58,77.55,NaN,NaN,97,0,4
1,2016,M-Palacio,40 a 60 m2,49,3268.58,77.55,NaN,NaN,97,0,0
2,2016,M-Palacio,60 a 80 m2,33,3268.58,77.55,NaN,NaN,97,0,1
3,2016,M-Palacio,80 a 100 m2,23,3268.58,77.55,NaN,NaN,97,0,2
4,2016,M-Palacio,over 100 m2,32,3268.58,77.55,NaN,NaN,97,0,3
...,...,...,...,...,...,...,...,...,...,...,...
3135,2019,Villaviciosa de Odón,under 40 m2,7,2589,96,NaN,NaN,156,3,4
3136,2019,Villaviciosa de Odón,40 a 60 m2,49,2589,96,NaN,NaN,156,3,0
3137,2019,Villaviciosa de Odón,60 a 80 m2,87,2589,96,NaN,NaN,156,3,1
3138,2019,Villaviciosa de Odón,80 a 100 m2,58,2589,96,NaN,NaN,156,3,2


In [150]:
df_past_mad['transactions']=pd.to_numeric(df_past_mad['transactions'])
df_past_mad['avgsize']=pd.to_numeric(df_past_mad['avgsize'])
df_past_mad['avg_eur_m2']=pd.to_numeric(df_past_mad['avg_eur_m2'])
df_past_mad['neighbourhood']=df_past_mad['neighbourhood'].astype('category')
df_past_mad['size']=df_past_mad['size'].astype('category')

In [268]:
df_past_mad.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3140 entries, 0 to 3139
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   year               3140 non-null   int64   
 1   neighbourhood      3140 non-null   category
 2   size               3140 non-null   category
 3   transactions       3140 non-null   int64   
 4   avg_eur_m2         3140 non-null   float64 
 5   avgsize            3140 non-null   float64 
 6   gdp                0 non-null      float64 
 7   unempl_rate        0 non-null      float64 
 8   neighbourhood_cat  3140 non-null   int64   
 9   year_cat           3140 non-null   int64   
 10  size_cat           3140 non-null   int64   
dtypes: category(2), float64(4), int64(5)
memory usage: 236.5 KB


In [117]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
#Encode Neighbourhood Column
label_enc_X = LabelEncoder()
X['neighbourhood'] = label_enc_X.fit_transform(X['neighbourhood'])
ct = ColumnTransformer([("neighbourhood", OneHotEncoder(), [0])], remainder = 'passthrough')
X = ct.fit_transform(X)

In [269]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

df_past_mad[['transactions', 'neighbourhood_cat','avgsize','year_cat','size_cat']] = scaler.fit_transform(df_past_mad[['transactions', 'neighbourhood_cat','avgsize','year_cat','size_cat']])

df_past_mad

,year,neighbourhood,size,transactions,avg_eur_m2,avgsize,gdp,unempl_rate,neighbourhood_cat,year_cat,size_cat
0,2016,M-Palacio,under 40 m2,0.041540,3268.58,0.386378,NaN,NaN,0.621795,0.0,1.00
1,2016,M-Palacio,40 a 60 m2,0.049645,3268.58,0.386378,NaN,NaN,0.621795,0.0,0.00
2,2016,M-Palacio,60 a 80 m2,0.033435,3268.58,0.386378,NaN,NaN,0.621795,0.0,0.25
3,2016,M-Palacio,80 a 100 m2,0.023303,3268.58,0.386378,NaN,NaN,0.621795,0.0,0.50
4,2016,M-Palacio,over 100 m2,0.032421,3268.58,0.386378,NaN,NaN,0.621795,0.0,0.75
...,...,...,...,...,...,...,...,...,...,...,...
3135,2019,Villaviciosa de Odón,under 40 m2,0.007092,2589.00,0.478302,NaN,NaN,1.000000,1.0,1.00
3136,2019,Villaviciosa de Odón,40 a 60 m2,0.049645,2589.00,0.478302,NaN,NaN,1.000000,1.0,0.00
3137,2019,Villaviciosa de Odón,60 a 80 m2,0.088146,2589.00,0.478302,NaN,NaN,1.000000,1.0,0.25
3138,2019,Villaviciosa de Odón,80 a 100 m2,0.058764,2589.00,0.478302,NaN,NaN,1.000000,1.0,0.50


In [355]:
df_past_mad.values

array([[2016, 'M-Palacio', 'under 40 m2', ..., 0.6217948717948718, 0.0,
        1.0],
       [2016, 'M-Palacio', '40 a 60 m2', ..., 0.6217948717948718, 0.0,
        0.0],
       [2016, 'M-Palacio', '60 a 80 m2', ..., 0.6217948717948718, 0.0,
        0.25],
       ...,
       [2019, 'Villaviciosa de Odón', '60 a 80 m2', ..., 1.0, 1.0, 0.25],
       [2019, 'Villaviciosa de Odón', '80 a 100 m2', ..., 1.0, 1.0, 0.5],
       [2019, 'Villaviciosa de Odón', 'over 100 m2', ..., 1.0, 1.0, 0.75]],
      dtype=object)

In [270]:
y=np.array(df_past_mad['avg_eur_m2'])
X= df_past_mad.drop(columns=['avg_eur_m2','size','year','gdp','unempl_rate','neighbourhood'], axis=1)

In [329]:
x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.35,random_state=40,shuffle=True)

In [330]:
lr=LinearRegression()


In [331]:
lr.fit(x_train,y_train)

LinearRegression()

In [179]:
from sklearn.metrics import mean_absolute_error, mean_squared_error,root,r2_score

In [332]:
predictions_lr= lr.predict(x_test)
#probabilities= lr.predict_proba(x_test)

In [333]:
rmse= mean_squared_error(y_test,predictions_lr,squared=False)
r2_confidence = r2_score(y_test,predictions_lr)
mae=mean_absolute_error(y_test,predictions_lr)
mse=mean_squared_error(y_test,predictions_lr)
print('RMSE: ', rmse)
print('R2 confidence: ', r2_confidence)
print('MAE: ', mae)
print('MSE: ', mse)

RMSE:  1149.4644035530005
R2 confidence:  0.31151132776263557
MAE:  908.6393221193313
MSE:  1321268.415035455


In [388]:
from sklearn.linear_model import Lasso

In [407]:
df_past_mad

,year,neighbourhood,size,transactions,avgsize,neighbourhood_cat,year_cat,size_cat,gdp,unempl_rate,avg_eur_m2
0,2016,M-Palacio,under 40 m2,0.041540,0.386378,0.621795,0.0,1.00,NaN,NaN,3268.58
1,2016,M-Palacio,40 a 60 m2,0.049645,0.386378,0.621795,0.0,0.00,NaN,NaN,3268.58
2,2016,M-Palacio,60 a 80 m2,0.033435,0.386378,0.621795,0.0,0.25,NaN,NaN,3268.58
3,2016,M-Palacio,80 a 100 m2,0.023303,0.386378,0.621795,0.0,0.50,NaN,NaN,3268.58
4,2016,M-Palacio,over 100 m2,0.032421,0.386378,0.621795,0.0,0.75,NaN,NaN,3268.58
...,...,...,...,...,...,...,...,...,...,...,...
3135,2019,Villaviciosa de Odón,under 40 m2,0.007092,0.478302,1.000000,1.0,1.00,NaN,NaN,2589.00
3136,2019,Villaviciosa de Odón,40 a 60 m2,0.049645,0.478302,1.000000,1.0,0.00,NaN,NaN,2589.00
3137,2019,Villaviciosa de Odón,60 a 80 m2,0.088146,0.478302,1.000000,1.0,0.25,NaN,NaN,2589.00
3138,2019,Villaviciosa de Odón,80 a 100 m2,0.058764,0.478302,1.000000,1.0,0.50,NaN,NaN,2589.00


In [406]:
df_past_mad=df_past_mad[['year', 'neighbourhood', 'size', 'transactions', 'avgsize','neighbourhood_cat', 'year_cat', 'size_cat', 'gdp', 'unempl_rate','avg_eur_m2']]

In [410]:

data_past=df_past_mad.values

In [411]:
data_past[:,3:-3]

array([[0.04154002026342452, 0.38637835683324195, 0.6217948717948718,
        0.0, 1.0],
       [0.04964539007092199, 0.38637835683324195, 0.6217948717948718,
        0.0, 0.0],
       [0.033434650455927056, 0.38637835683324195, 0.6217948717948718,
        0.0, 0.25],
       ...,
       [0.08814589665653497, 0.4783020278013053, 1.0, 1.0, 0.25],
       [0.05876393110435664, 0.4783020278013053, 1.0, 1.0, 0.5],
       [0.09017223910840932, 0.4783020278013053, 1.0, 1.0, 0.75]],
      dtype=object)

In [412]:
#dataframe = read_csv(url, header=None)
X, y = data_past[:,3:-3], data_past[:,-1]
# define model
#lass= Lasso(alpha=1.0)
            

In [414]:
# fit model
ls.fit(X, y)
# define new data
row = [0.08,0.5,1,1,0.5]
# make a prediction
yhat = ls.predict([row])
# summarize prediction
print('Predicted: %.3f' % yhat)

Predicted: 3380.894


In [419]:
model_lass=ls.fit(X,y)

model_lass.score
#model_lass.best_params_

<bound method RegressorMixin.score of Lasso(alpha=0.5)>

In [415]:
rmse= mean_squared_error(y,yhat,squared=False)
r2_confidence = r2_score(y,yhat)
mae=mean_absolute_error(y,yhat)
mse=mean_squared_error(y,yhat)
print('RMSE: ', rmse)
print('R2 confidence: ', r2_confidence)
print('MAE: ', mae)
print('MSE: ', mse)

ValueError: Found input variables with inconsistent numbers of samples: [3140, 1]

In [392]:
ls=Lasso(alpha=.5)
ls.fit(x_train,y_train)


Lasso(alpha=0.5)

In [347]:
predictions_ls= ls.predict(x_test)

In [348]:
rmse= mean_squared_error(y_test,predictions_ls,squared=False)
r2_confidence = r2_score(y_test,predictions_ls)
mae=mean_absolute_error(y_test,predictions_ls)
mse=mean_squared_error(y_test,predictions_ls)
print('RMSE: ', rmse)
print('R2 confidence: ', r2_confidence)
print('MAE: ', mae)
print('MSE: ', mse)

RMSE:  1149.8056165225978
R2 confidence:  0.3111025180051785
MAE:  909.3773982137133
MSE:  1322052.9557869113


In [213]:
from sklearn.linear_model import Ridge

In [352]:
rd=Ridge(alpha=0.9)
rd.fit(x_train,y_train)

Ridge(alpha=0.9)

In [353]:
predictions_rd=rd.predict(x_test)

In [354]:
rmse= mean_squared_error(y_test,predictions_rd,squared=False)
r2_confidence = r2_score(y_test,predictions_rd)
mae=mean_absolute_error(y_test,predictions_rd)
mse=mean_squared_error(y_test,predictions_rd)
print('RMSE: ', rmse)
print('R2 confidence: ', r2_confidence)
print('MAE: ', mae)
print('MSE: ', mse)

RMSE:  1150.393156979477
R2 confidence:  0.31039829713751643
MAE:  910.1366273948817
MSE:  1323404.4156252078


In [284]:
df_past_mad['prediction']=predictions_rd

ValueError: Length of values does not match length of index

In [287]:
len(predictions_rd)

3140